# Manipulate Frame coordinates

In [1]:
### different ways to expose to `numpy`
import numpy as np
import pytraj.io as io

traj = io.load("../tests/data/md1_prod.Tc5b.x", "../tests/data/Tc5b.top") 
# extract only first frame
frame = traj[0]
print (frame)

<Frame with 304 atoms>


In [2]:
arr0 = np.asarray(frame[:]) # need to use `[:]`
arr1 = np.asarray(frame.buffer2d) # 
xyz = frame.xyz
print (type(xyz), type(arr0), type(arr1))

# try to change coors of 1st atom
xyz[0] = [1.0, 2.0, 3.0]
print (xyz[0], arr0[0], arr1[0], frame[0], frame.xyz[0])

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
[ 1.  2.  3.] [ 1.  2.  3.] [ 1.  2.  3.] [ 1.  2.  3.] [ 1.  2.  3.]


In [9]:
# make sure to use [:] if changing coords

xyz = np.random.rand(frame.n_atoms*3).reshape((frame.n_atoms, 3))
print (xyz[0])
print (frame.xyz[0]) # updating `xyz` does not update frame.xyz because we just `point` xyz to different array location

print ("")
print ("lets try again by using `[:]`. This mean 'please update coords that xyz points to'")
# need to create memory first
xyz = frame.xyz
xyz[:] = np.random.rand(frame.n_atoms*3).reshape((frame.n_atoms, 3))
print (xyz[0])
print (frame.xyz[0]) # updating `xyz` will upate frame.xyz too

# then you're free to use pytraj/cpptraj' Actions 
import pytraj.common_actions as pyca
vec = pyca.calc_COM(traj=frame, command="@CA @CB", top=traj.top)
# could use `pyca.calc_COM(frame, "@CA @CB", traj.top)
print ("")
print ("vector center of mass")
print (vec.to_ndarray())

[ 0.0541192   0.65207231  0.40998237]
[ 0.65099735  0.54863899  0.80845828]

lets try again by using `[:]`. This mean 'please update coords that xyz points to'
[ 0.93723133  0.76371033  0.570445  ]
[ 0.93723133  0.76371033  0.570445  ]

vector center of mass
[[ 0.42094255  0.6064389   0.50495557]]


#### what should I use, xyz or frame.xyz?

In [4]:
# definitely you should use `xyz` if you want to use fast numpy calculation. Let's see the time

%timeit xyz[:] 
%timeit frame.xyz[:] # is about 32 times slower calling xyz[:]

The slowest run took 24.31 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 411 ns per loop
The slowest run took 5.43 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 14.6 µs per loop


In [5]:
# this is the behind the scene: Whenever calling `frame.xyz`, there is one more casting step `np.asarray(frame.buffe2d[:]